# crop

預測bounding boxes並切割資料夾中所有細胞並存入資料夾中

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("E:/together/tiffany/Mask_RCNN-master")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from samples.balloon import balloon

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to Ballon trained weights
# You can download this file from the Releases page
# https://github.com/matterport/Mask_RCNN/releases
#BALLON_WEIGHTS_PATH = "/path/to/mask_rcnn_balloon.h5"  # TODO: update this path

Using TensorFlow backend.


## Configurations

In [2]:
config = balloon.BalloonConfig()
BALLOON_DIR = os.path.join(ROOT_DIR, "samples\\nucleus_crop\\datasets")

In [3]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0 #7/3解決No instances to display
    
config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.004
LOSS_WEIGHTS                   {'mrcnn_mask_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE           

## Notebook Preferences

In [4]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [5]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Validation Dataset

In [6]:
# Load validation dataset
dataset = balloon.BalloonDataset()
dataset.load_balloon(BALLOON_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Images: 6
Classes: ['BG', 'balloon']


## Load Model

In [7]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [8]:
# Set path to balloon weights file

# Download file from the Releases page and set its path
# https://github.com/matterport/Mask_RCNN/releases
# weights_path = "/path/to/mask_rcnn_balloon.h5"

# Or, load the last model you trained
#weights_path = "E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\mask_rcnn_nucleus_0040.h5"
weights_path = "E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\mask_rcnn_balloon_0010.h5"


# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  E:\together\tiffany\Mask_RCNN-master\samples\nucleus_crop\mask_rcnn_balloon_0010.h5


## Run Detection

In [9]:
#測試一張照片
import skimage.io
IMAGE_DIR="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\datasets\\ANA_picked\\Coarse speckled"
# Load a random image from the images folder

file_names = next(os.walk(IMAGE_DIR))[2]
#print(file_names) #發現是一個list
#print(file_names[1])

"""
for filenames in os.listdir(IMAGE_DIR):              #listdir的参数是文件夹的路径
    print (filenames)  
    print(type(filenames))
"""
#統計資料夾中有幾張圖片

quantity=(len([name for name in os.listdir(IMAGE_DIR) if os.path.isfile(os.path.join(IMAGE_DIR, name))]))
print(quantity)



273


In [10]:
#將資料夾中所有圖片都預測出來並把bounding boxes存到同一個資料夾

from PIL import Image

pre_quantity_list=[] 
final=[]

for i in range(0, quantity):
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, filenames))
    image = skimage.io.imread(os.path.join(IMAGE_DIR, file_names[i]))
    
    #print(len(images))

    # Run detection
    results = model.detect([image], verbose=1)

    # Visualize results

    r = results[0]

    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            #dataset.class_names, r['scores'])
        
    #把這一張圖片的細胞數量存入pre_quantity變數中    
    pre_quantity=len(r['scores']) 
    #print(pre_quantity) 
    
    
    #每張預測出細胞數存入list中     
    pre_quantity_list.append(pre_quantity)
    
    #累計預測出的細胞數量好讓儲存好的細胞依序在資料夾中編號
    total=0
    for q in range(0,i):    
        total=total + pre_quantity_list[q]
        #print(total)
        
    
    #做此張圖片的crop
    im = Image.open(os.path.join(IMAGE_DIR, file_names[i])) #叫出現在正在處理的這張圖片來進行裁剪
    
    
    for j in range(0, len(r['scores'])):
        
        region = im.crop((r['rois'][j][1], r['rois'][j][0], r['rois'][j][3], r['rois'][j][2])) #crop(x1,y1,x2,y2) -->(x1,y1)為左上角座標 (x2,y2)為右下角座標<即寬長>
        
    
    
        #裁減出來的圖片存入指定資料夾,下張圖片繼續接續
        path_save="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\classifier_datas\\Coarse speckled"
        
        if i==0:
            name = str(j) + ".jpeg"
            region.save(os.path.join(path_save,name))
        else:
            name = str(total+j) + ".jpeg" #承接上一次的數量接著下去存
            region.save(os.path.join(path_save,name))
    
    
    
    
        
        
 #print(pre_quantity_list)

    #換下一張圖片
    i=i+1


Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  213.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   95.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  181.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   61.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  226.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  106.2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  158.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   39.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  122.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:    1.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  172.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   52.2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  149.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   28.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  147.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   26.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  147.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   29.2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  188.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   66.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  198.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   78.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  194.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   75.2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  132.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   14.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  166.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   43.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  193.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   71.2

...

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  125.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:    6.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  163.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   41.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  103.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -16.8

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  233.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  111.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  192.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   69.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  150.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   29.2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  156.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   36.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  182.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   63.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  161.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   39.2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  126.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   -3.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  135.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:    7.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  138.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   17.2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  159.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   35.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  235.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  102.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  205.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   69.2

In [11]:
#測試一張照片
import skimage.io
IMAGE_DIR="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\datasets\\ANA_picked\\Cytoplasmic"
# Load a random image from the images folder

file_names = next(os.walk(IMAGE_DIR))[2]
#print(file_names) #發現是一個list
#print(file_names[1])

"""
for filenames in os.listdir(IMAGE_DIR):              #listdir的参数是文件夹的路径
    print (filenames)  
    print(type(filenames))
"""
#統計資料夾中有幾張圖片

quantity=(len([name for name in os.listdir(IMAGE_DIR) if os.path.isfile(os.path.join(IMAGE_DIR, name))]))
print(quantity)



32


In [12]:
from PIL import Image

pre_quantity_list=[] 
final=[]

for i in range(0, quantity):
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, filenames))
    image = skimage.io.imread(os.path.join(IMAGE_DIR, file_names[i]))
    
    #print(len(images))

    # Run detection
    results = model.detect([image], verbose=1)

    # Visualize results

    r = results[0]

    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            #dataset.class_names, r['scores'])
        
    #把這一張圖片的細胞數量存入pre_quantity變數中    
    pre_quantity=len(r['scores']) 
    #print(pre_quantity) 
    
    
    #每張預測出細胞數存入list中     
    pre_quantity_list.append(pre_quantity)
    
    #累計預測出的細胞數量好讓儲存好的細胞依序在資料夾中編號
    total=0
    for q in range(0,i):    
        total=total + pre_quantity_list[q]
        #print(total)
        
    
    #做此張圖片的crop
    im = Image.open(os.path.join(IMAGE_DIR, file_names[i])) #叫出現在正在處理的這張圖片來進行裁剪
    
    
    for j in range(0, len(r['scores'])):
        
        region = im.crop((r['rois'][j][1], r['rois'][j][0], r['rois'][j][3], r['rois'][j][2])) #crop(x1,y1,x2,y2) -->(x1,y1)為左上角座標 (x2,y2)為右下角座標<即寬長>
        
    
    
        #裁減出來的圖片存入指定資料夾,下張圖片繼續接續
        path_save="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\classifier_datas\\Cytoplasmic"
        
        if i==0:
            name = str(j) + ".jpeg"
            region.save(os.path.join(path_save,name))
        else:
            name = str(total+j) + ".jpeg" #承接上一次的數量接著下去存
            region.save(os.path.join(path_save,name))
    
    
    
    
        
        
 #print(pre_quantity_list)

    #換下一張圖片
    i=i+1


Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  169.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -24.70000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  215.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   18.30000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  197.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   74.2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  146.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   24.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:   95.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -38.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  114.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -32.8

ValueError: Could not find a format to read the specified file in mode 'i'

In [13]:
#測試一張照片
import skimage.io
IMAGE_DIR="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\datasets\\ANA_picked\\Diffused"
# Load a random image from the images folder

file_names = next(os.walk(IMAGE_DIR))[2]
#print(file_names) #發現是一個list
#print(file_names[1])

"""
for filenames in os.listdir(IMAGE_DIR):              #listdir的参数是文件夹的路径
    print (filenames)  
    print(type(filenames))
"""
#統計資料夾中有幾張圖片

quantity=(len([name for name in os.listdir(IMAGE_DIR) if os.path.isfile(os.path.join(IMAGE_DIR, name))]))
print(quantity)

from PIL import Image

pre_quantity_list=[] 
final=[]

for i in range(0, quantity):
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, filenames))
    image = skimage.io.imread(os.path.join(IMAGE_DIR, file_names[i]))
    
    #print(len(images))

    # Run detection
    results = model.detect([image], verbose=1)

    # Visualize results

    r = results[0]

    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            #dataset.class_names, r['scores'])
        
    #把這一張圖片的細胞數量存入pre_quantity變數中    
    pre_quantity=len(r['scores']) 
    #print(pre_quantity) 
    
    
    #每張預測出細胞數存入list中     
    pre_quantity_list.append(pre_quantity)
    
    #累計預測出的細胞數量好讓儲存好的細胞依序在資料夾中編號
    total=0
    for q in range(0,i):    
        total=total + pre_quantity_list[q]
        #print(total)
        
    
    #做此張圖片的crop
    im = Image.open(os.path.join(IMAGE_DIR, file_names[i])) #叫出現在正在處理的這張圖片來進行裁剪
    
    
    for j in range(0, len(r['scores'])):
        
        region = im.crop((r['rois'][j][1], r['rois'][j][0], r['rois'][j][3], r['rois'][j][2])) #crop(x1,y1,x2,y2) -->(x1,y1)為左上角座標 (x2,y2)為右下角座標<即寬長>
        
    
    
        #裁減出來的圖片存入指定資料夾,下張圖片繼續接續
        path_save="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\classifier_datas\\Diffused"
        
        if i==0:
            name = str(j) + ".jpeg"
            region.save(os.path.join(path_save,name))
        else:
            name = str(total+j) + ".jpeg" #承接上一次的數量接著下去存
            region.save(os.path.join(path_save,name))
    
    
    
    
        
        
 #print(pre_quantity_list)

    #換下一張圖片
    i=i+1


33
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  167.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   46.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  140.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   20.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  112.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   -

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  115.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   -4.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  166.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   44.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  156.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   34.2

In [14]:
#測試一張照片
import skimage.io
IMAGE_DIR="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\datasets\\ANA_picked\\Discrete speckled"
# Load a random image from the images folder

file_names = next(os.walk(IMAGE_DIR))[2]
#print(file_names) #發現是一個list
#print(file_names[1])

"""
for filenames in os.listdir(IMAGE_DIR):              #listdir的参数是文件夹的路径
    print (filenames)  
    print(type(filenames))
"""
#統計資料夾中有幾張圖片

quantity=(len([name for name in os.listdir(IMAGE_DIR) if os.path.isfile(os.path.join(IMAGE_DIR, name))]))
print(quantity)

from PIL import Image

pre_quantity_list=[] 
final=[]

for i in range(0, quantity):
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, filenames))
    image = skimage.io.imread(os.path.join(IMAGE_DIR, file_names[i]))
    
    #print(len(images))

    # Run detection
    results = model.detect([image], verbose=1)

    # Visualize results

    r = results[0]

    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            #dataset.class_names, r['scores'])
        
    #把這一張圖片的細胞數量存入pre_quantity變數中    
    pre_quantity=len(r['scores']) 
    #print(pre_quantity) 
    
    
    #每張預測出細胞數存入list中     
    pre_quantity_list.append(pre_quantity)
    
    #累計預測出的細胞數量好讓儲存好的細胞依序在資料夾中編號
    total=0
    for q in range(0,i):    
        total=total + pre_quantity_list[q]
        #print(total)
        
    
    #做此張圖片的crop
    im = Image.open(os.path.join(IMAGE_DIR, file_names[i])) #叫出現在正在處理的這張圖片來進行裁剪
    
    
    for j in range(0, len(r['scores'])):
        
        region = im.crop((r['rois'][j][1], r['rois'][j][0], r['rois'][j][3], r['rois'][j][2])) #crop(x1,y1,x2,y2) -->(x1,y1)為左上角座標 (x2,y2)為右下角座標<即寬長>
        
    
    
        #裁減出來的圖片存入指定資料夾,下張圖片繼續接續
        path_save="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\classifier_datas\\Discrete speckled"
        
        if i==0:
            name = str(j) + ".jpeg"
            region.save(os.path.join(path_save,name))
        else:
            name = str(total+j) + ".jpeg" #承接上一次的數量接著下去存
            region.save(os.path.join(path_save,name))
    
    
    
    
        
        
 #print(pre_quantity_list)

    #換下一張圖片
    i=i+1


28
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  148.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   28.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  151.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   29.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  105.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  194.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   62.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  152.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   27.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  183.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   58.2

In [15]:
#測試一張照片
import skimage.io
IMAGE_DIR="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\datasets\\ANA_picked\\Fine speckled"
# Load a random image from the images folder

file_names = next(os.walk(IMAGE_DIR))[2]
#print(file_names) #發現是一個list
#print(file_names[1])

"""
for filenames in os.listdir(IMAGE_DIR):              #listdir的参数是文件夹的路径
    print (filenames)  
    print(type(filenames))
"""
#統計資料夾中有幾張圖片

quantity=(len([name for name in os.listdir(IMAGE_DIR) if os.path.isfile(os.path.join(IMAGE_DIR, name))]))
print(quantity)

from PIL import Image

pre_quantity_list=[] 
final=[]

for i in range(0, quantity):
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, filenames))
    image = skimage.io.imread(os.path.join(IMAGE_DIR, file_names[i]))
    
    #print(len(images))

    # Run detection
    results = model.detect([image], verbose=1)

    # Visualize results

    r = results[0]

    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            #dataset.class_names, r['scores'])
        
    #把這一張圖片的細胞數量存入pre_quantity變數中    
    pre_quantity=len(r['scores']) 
    #print(pre_quantity) 
    
    
    #每張預測出細胞數存入list中     
    pre_quantity_list.append(pre_quantity)
    
    #累計預測出的細胞數量好讓儲存好的細胞依序在資料夾中編號
    total=0
    for q in range(0,i):    
        total=total + pre_quantity_list[q]
        #print(total)
        
    
    #做此張圖片的crop
    im = Image.open(os.path.join(IMAGE_DIR, file_names[i])) #叫出現在正在處理的這張圖片來進行裁剪
    
    
    for j in range(0, len(r['scores'])):
        
        region = im.crop((r['rois'][j][1], r['rois'][j][0], r['rois'][j][3], r['rois'][j][2])) #crop(x1,y1,x2,y2) -->(x1,y1)為左上角座標 (x2,y2)為右下角座標<即寬長>
        
    
    
        #裁減出來的圖片存入指定資料夾,下張圖片繼續接續
        path_save="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\classifier_datas\\Fine speckled"
        
        if i==0:
            name = str(j) + ".jpeg"
            region.save(os.path.join(path_save,name))
        else:
            name = str(total+j) + ".jpeg" #承接上一次的數量接著下去存
            region.save(os.path.join(path_save,name))
    
    
    
    
        
        
 #print(pre_quantity_list)

    #換下一張圖片
    i=i+1


34
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  119.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   -0.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:   97.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -49.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  101.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -3

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  122.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -43.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  129.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -19.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  136.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -14.8

ValueError: Could not find a format to read the specified file in mode 'i'

In [16]:
#測試一張照片
import skimage.io
IMAGE_DIR="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\datasets\\ANA_picked\\Nucleolar"
# Load a random image from the images folder

file_names = next(os.walk(IMAGE_DIR))[2]
#print(file_names) #發現是一個list
#print(file_names[1])

"""
for filenames in os.listdir(IMAGE_DIR):              #listdir的参数是文件夹的路径
    print (filenames)  
    print(type(filenames))
"""
#統計資料夾中有幾張圖片

quantity=(len([name for name in os.listdir(IMAGE_DIR) if os.path.isfile(os.path.join(IMAGE_DIR, name))]))
print(quantity)

from PIL import Image

pre_quantity_list=[] 
final=[]

for i in range(0, quantity):
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, filenames))
    image = skimage.io.imread(os.path.join(IMAGE_DIR, file_names[i]))
    
    #print(len(images))

    # Run detection
    results = model.detect([image], verbose=1)

    # Visualize results

    r = results[0]

    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            #dataset.class_names, r['scores'])
        
    #把這一張圖片的細胞數量存入pre_quantity變數中    
    pre_quantity=len(r['scores']) 
    #print(pre_quantity) 
    
    
    #每張預測出細胞數存入list中     
    pre_quantity_list.append(pre_quantity)
    
    #累計預測出的細胞數量好讓儲存好的細胞依序在資料夾中編號
    total=0
    for q in range(0,i):    
        total=total + pre_quantity_list[q]
        #print(total)
        
    
    #做此張圖片的crop
    im = Image.open(os.path.join(IMAGE_DIR, file_names[i])) #叫出現在正在處理的這張圖片來進行裁剪
    
    
    for j in range(0, len(r['scores'])):
        
        region = im.crop((r['rois'][j][1], r['rois'][j][0], r['rois'][j][3], r['rois'][j][2])) #crop(x1,y1,x2,y2) -->(x1,y1)為左上角座標 (x2,y2)為右下角座標<即寬長>
        
    
    
        #裁減出來的圖片存入指定資料夾,下張圖片繼續接續
        path_save="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\classifier_datas\\Nucleolar"
        
        if i==0:
            name = str(j) + ".jpeg"
            region.save(os.path.join(path_save,name))  
        else:
            name = str(total+j) + ".jpeg" #承接上一次的數量接著下去存
            region.save(os.path.join(path_save,name))
    
    
    
    
        
        
 #print(pre_quantity_list)

    #換下一張圖片
    i=i+1


60
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:   97.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -54.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  112.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -49.70000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  112.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -3

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  123.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:    4.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  152.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   29.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  150.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   30.2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  105.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -17.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  101.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -20.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  151.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   30.2

ValueError: Could not find a format to read the specified file in mode 'i'

In [17]:

#測試一張照片
import skimage.io
IMAGE_DIR="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\datasets\\ANA_picked\\Peripheral"
# Load a random image from the images folder

file_names = next(os.walk(IMAGE_DIR))[2]
#print(file_names) #發現是一個list
#print(file_names[1])

"""
for filenames in os.listdir(IMAGE_DIR):              #listdir的参数是文件夹的路径
    print (filenames)  
    print(type(filenames))
"""
#統計資料夾中有幾張圖片

quantity=(len([name for name in os.listdir(IMAGE_DIR) if os.path.isfile(os.path.join(IMAGE_DIR, name))]))
print(quantity)

from PIL import Image

pre_quantity_list=[] 
final=[]

for i in range(0, quantity):
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
    #image = skimage.io.imread(os.path.join(IMAGE_DIR, filenames))
    image = skimage.io.imread(os.path.join(IMAGE_DIR, file_names[i]))
    
    #print(len(images))

    # Run detection
    results = model.detect([image], verbose=1)

    # Visualize results

    r = results[0]

    #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            #dataset.class_names, r['scores'])
        
    #把這一張圖片的細胞數量存入pre_quantity變數中    
    pre_quantity=len(r['scores']) 
    #print(pre_quantity) 
    
    
    #每張預測出細胞數存入list中     
    pre_quantity_list.append(pre_quantity)
    
    #累計預測出的細胞數量好讓儲存好的細胞依序在資料夾中編號
    total=0
    for q in range(0,i):    
        total=total + pre_quantity_list[q]
        #print(total)
        
    
    #做此張圖片的crop
    im = Image.open(os.path.join(IMAGE_DIR, file_names[i])) #叫出現在正在處理的這張圖片來進行裁剪
    
    
    for j in range(0, len(r['scores'])):
        
        region = im.crop((r['rois'][j][1], r['rois'][j][0], r['rois'][j][3], r['rois'][j][2])) #crop(x1,y1,x2,y2) -->(x1,y1)為左上角座標 (x2,y2)為右下角座標<即寬長>
        
    
    
        #裁減出來的圖片存入指定資料夾,下張圖片繼續接續
        path_save="E:\\together\\tiffany\\Mask_RCNN-master\\samples\\nucleus_crop\\classifier_datas\\Peripheral"
        
        if i==0:
            name = str(j) + ".jpeg"
            region.save(os.path.join(path_save,name))
        else:
            name = str(total+j) + ".jpeg" #承接上一次的數量接著下去存
            region.save(os.path.join(path_save,name))
    
    
    
    
        
        
 #print(pre_quantity_list)

    #換下一張圖片
    i=i+1

75
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  184.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   63.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  121.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   -6.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  100.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  177.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   56.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  188.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   70.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  182.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   62.2

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:   69.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -68.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:   87.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -48.80000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:   93.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  -49.8

Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  168.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   44.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  146.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   24.20000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3136.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2352, 3136, 3)       min:    0.00000  max:  151.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   27.2

In [20]:
print(r['rois'])

print(r['rois'][0][0])
print(r['rois'][1][0])
print(r['scores'])
len(r['scores']) #測出來的數量 #若只有一個遮罩會出現問題

[[1826  105 1998  242]
 [1001 2686 1161 2887]
 [1332 2506 1567 2651]
 [1264 2787 1437 2965]
 [1528 2657 1724 2789]
 [1430 1964 1643 2100]
 [1382 2291 1566 2474]
 [ 692 2802  883 3006]
 [ 503  631  667  859]
 [ 617 1229  820 1364]
 [ 326 1676  473 1860]
 [ 803 1854  996 2014]
 [2059 1047 2246 1237]
 [1073  228 1222  416]
 [1204 1805 1364 1994]
 [ 692 2196  909 2453]
 [ 855   94  991  264]
 [1667 1765 1860 1922]
 [ 211 2611  382 2763]
 [1714 2174 1862 2369]
 [ 105  146  309  274]
 [1658  616 1799  800]
 [ 692  787  842  936]
 [1542 2081 1724 2231]
 [1477 1485 1634 1639]
 [ 232 1186  376 1354]
 [ 801  490  940  651]
 [ 625 2049  837 2223]
 [2165 1799 2348 1973]
 [1573 1293 1737 1474]
 [1277 1667 1445 1825]
 [ 959 1080 1110 1247]
 [ 295 2328  425 2514]
 [1032 2212 1187 2371]
 [ 147  743  325  924]
 [2201 1331 2352 1508]
 [ 727  630  893  762]
 [1613  903 1768 1063]
 [ 666  314  858  453]
 [ 748 1409  961 1603]
 [  43  373  198  537]
 [1762 1456 1919 1606]
 [ 206 2776  371 2909]
 [1086 1919

56

In [124]:
print(r['rois'])

print(r['rois'][0][0])
print(r['scores'])

[[342 583 753 923]]
342
[0.99995816]
